In [1]:
import os
import pandas as pd
import cv2
# from skimage.measure import compare_ssim
from skimage.metrics import structural_similarity
from matplotlib import pyplot as plt

In [7]:
classes = sorted([i for i in os.listdir('./original_pictures') if i[0] is not '.' if os.path.isdir('./original_pictures/'+i)])
classes

<>:1: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:1: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-7-05a6e4c77933>:1: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  classes = sorted([i for i in os.listdir('./original_pictures') if i[0] is not '.' if os.path.isdir('./original_pictures/'+i)])


['01_nakseong',
 '02_scuba',
 '03_osteria',
 '04_houskewer',
 '05_gravy',
 '06_jungga',
 '07_edonggan',
 '08_hwanyeo',
 '09_badarang',
 '10_sushimura',
 '11_ugo',
 '12_kingsjokbal',
 '13_kajung',
 '14_phbanlim',
 '15_pyeongnam',
 '16_tomatilo',
 '17_jobangnakji',
 '18_samohbokjib',
 '19_marado',
 'unseen']

In [4]:
# 안나올때까지 돌려야됨
# 동일한 크기를 갖는 파일 목록
dup_photo = pd.DataFrame(columns=['filename', 'size'])
for i in classes:
    photo_list = os.listdir('./original_pictures/'+i)
    photo_size = [os.path.getsize('./original_pictures/'+i+'/'+x) for x in photo_list]
    photo = pd.DataFrame({'filename':photo_list, 'size':photo_size})
    if len(photo) != len(set(photo_size)):#중복된 사이즈가 있을때
        temp = pd.DataFrame(photo['size'].value_counts()>1)
        dup_size = list(temp[temp['size']==True].index)
        for j in range(len(dup_size)):
            dup_photo = pd.concat([dup_photo, photo[photo['size']==dup_size[j]]])
dup_photo.reset_index(drop=True, inplace=True)
dup_photo['path'] = './original_pictures/'+dup_photo.filename.str.split('_').str[0]+'_'+dup_photo.filename.str.split('_').str[1]+'/'

uniq = dup_photo[['size', 'path']].drop_duplicates().reset_index(drop=True)
uniq.head()

#중복제거
for i in range(len(uniq)):
    temp = dup_photo[(dup_photo['size']==uniq['size'][i]) & (dup_photo['path']==uniq['path'][i])].reset_index(drop=True)
    img0 = cv2.imread(temp['path'][0]+temp['filename'][0])
    img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
    img_gray0 = cv2.cvtColor(img0, cv2.COLOR_RGB2GRAY)
    img1 = cv2.imread(temp['path'][1]+temp['filename'][1])
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img_gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    if img_gray0.shape == img_gray1.shape:
        score, diff = structural_similarity(img_gray0, img_gray1, full=True)
        if score > 0.99:
            fig = plt.figure()
            rows, cols = 1, 2
            
            ax1 = fig.add_subplot(rows,cols,1)
            ax1.imshow(img0)
            ax1.set_title(temp['filename'][0])
            ax1.axis('off')
            
            ax2=fig.add_subplot(rows,cols,2)
            ax2.imshow(img1)
            ax2.set_title(temp['filename'][1])
            ax2.axis('off')
            
            plt.show()
            print('사진이 중복되어, 오른쪽 그림을 삭제합니다.')
            os.remove(temp['path'][1]+temp['filename'][1])

In [8]:
#이름 변경(class_이름_사진번호)
for store in classes:
    for i, file in enumerate(os.listdir('./original_pictures/'+store)):
        os.rename('./original_pictures/'+store+'/'+file, './original_pictures/'+store+'/'+store+'_{0:05d}.jpg'.format(i+1))
    for i, file in enumerate(os.listdir('./original_pictures/'+store)):
        os.rename('./original_pictures/'+store+'/'+file, './original_pictures/'+store+'/'+store+'_{0:04d}.jpg'.format(i+1))

In [9]:
#class별 사진 수
print('Class별 사진 수')
for i in classes: 
    print(f'{i} : {len(os.listdir("./original_pictures/"+i))}')

Class별 사진 수
01_nakseong : 360
02_scuba : 310
03_osteria : 337
04_houskewer : 332
05_gravy : 310
06_jungga : 331
07_edonggan : 336
08_hwanyeo : 352
09_badarang : 333
10_sushimura : 393
11_ugo : 420
12_kingsjokbal : 319
13_kajung : 344
14_phbanlim : 162
15_pyeongnam : 396
16_tomatilo : 483
17_jobangnakji : 413
18_samohbokjib : 382
19_marado : 594
unseen : 318
